In [1]:
import pickle
import pandas as pd
import psycopg2 as pg
import geopandas as gpd

from Data_getter.get_data import get_buildings, get_service
from Blocks_getter.blocks_getter import BlocksModel
from Provision_getter.provision_getter import ProvisionModel
from Balancer.balancer import MasterPlan

/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/h8/0wmx2zx90bn60jw8zc1r4qjc0000gn/T/ipykernel_3970/4095684131.py:4: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (h

In [2]:
GROUPS = ["schools", "kindergartens"]

engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
city_crs = 32636
city_id = 1

service_type = "schools"

matrix = pd.read_pickle("/Users/mvin/Desktop/mp_dev/MASTERPLAN/Data/MMG_SPB.pkl")
engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
# buildings_gdf = get_buildings(engine=engine, city_crs=city_crs, city_id=city_id)
# service_gdf = get_service(engine=engine, city_crs=city_crs, city_id=city_id, service_type=service_type)
# blocks_gdf = BlocksModel(city_name='Санкт-Петербург', city_admin_level=5, city_crs=city_crs).get_blocks()
buildings_gdf = gpd.read_parquet("./output_data/buildings.parquet")
blocks_gdf = gpd.read_parquet("./output_data/blocks.parquet")

services_accessabilities = {
    "kindergartens": 4,
    "schools": 7,
    "universities": 60,
    "hospitals": 60,
    "policlinics": 13,
    "theaters": 60,
    "cinemas": 60,
    "cafes": 30,
    "bakeries": 30,
    "fastfoods": 30,
    "music_school": 30,
    "sportgrounds": 7,
    "swimming_pools": 30,
    "conveniences": 8,
    "recreational_areas": 25,
    "pharmacies": 7,
    "playgrounds": 2,
    "supermarkets": 30,
}

standard_dict = {
    "kindergartens": 61,
    "schools": 120,
    "universities": 13,
    "hospitals": 9,
    "policlinics": 27,
    "theaters": 5,
    "cinemas": 10,
    "cafes": 72,
    "bakeries": 72,
    "fastfoods": 72,
    "music_school": 8,
    "sportgrounds": 15,
    "swimming_pools": 50,
    "conveniences": 90,
    "recreational_areas": 5000,
    "pharmacies": 50,
    "playgrounds": 550,
    "supermarkets": 992,
}


for service_type in GROUPS:
    service_gdf = get_service(engine=engine, city_crs=city_crs, city_id=city_id, service_type=service_type)

    model = ProvisionModel(
        SUB_GROUP=service_type,
        blocks=blocks_gdf,
        matrix=matrix,
        buildings=buildings_gdf,
        standard=standard_dict[service_type],
        ACCS_TIME=services_accessabilities[service_type],
        service_gdf=service_gdf,
    )

    if service_type == "schools":
        school = model.run()

    elif service_type == "kindergartens":
        kindergarten = model.run()

    # elif service_type == 'recreational_areas':
    #     greening = model.run()

greening = gpd.read_parquet("./output_data/recreational_areas.parquet")
# prov_data = blocks_gdf.merge(school[['id', 'population_unprov_schools']], left_on='id', right_on='id').merge(kindergarten[['id', 'population_unprov_kindergartens']],
# left_on='id', right_on='id').merge(greening[['id', 'population_unprov_recreational_areas']], left_on='id', right_on='id')

/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


  0%|          | 0/605 [00:00<?, ?it/s]

количество кварталов c сервисом schools: 51
количество жилых кварталов: 1135
количество кварталов всего: 1609
количество кварталов c ошибкой: 0


/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


  0%|          | 0/691 [00:00<?, ?it/s]

количество кварталов c сервисом kindergartens: 29
количество жилых кварталов: 765
количество кварталов всего: 900
количество кварталов c ошибкой: 0


In [3]:
school.loc[school[f"population_unprov_schools"] > 0]

,id,geometry,provision_schools,id_schools,population_prov_schools,population_unprov_schools,population
566,566,"POLYGON ((320918.941 6654703.825, 320918.860 6...",25,327,297,861,1159
571,571,"POLYGON ((320624.168 6654912.212, 320624.085 6...",0,0,0,811,811
782,782,"POLYGON ((324402.745 6643448.134, 325085.481 6...",81,782,6733,1566,8300
991,991,"POLYGON ((331530.345 6664441.716, 331530.553 6...",53,985,340,300,641
997,997,"POLYGON ((331431.154 6664712.267, 331427.433 6...",0,0,0,26,26
...,...,...,...,...,...,...,...
5916,5916,"POLYGON ((364484.991 6632347.945, 364404.695 6...",29,5916,1466,3550,5017
5925,5925,"POLYGON ((364022.684 6633357.696, 364022.750 6...",57,5925,6716,5066,11783
5988,5988,"POLYGON ((360824.109 6651704.475, 360824.113 6...",64,5980,2363,1309,3672
6004,6004,"POLYGON ((365734.620 6623835.369, 365726.015 6...",70,6005,5192,2212,7405


In [4]:
school.loc[school[f"provision_schools"] == 100]

,id,geometry,provision_schools,id_schools,population_prov_schools,population_unprov_schools,population
76,76,"POLYGON ((316860.021 6677638.139, 316860.091 6...",100,365,44,0,44
83,83,"POLYGON ((316643.101 6677603.989, 316648.578 6...",100,83,108,0,108
84,84,"POLYGON ((317157.718 6677842.836, 317109.403 6...",100,83,9,0,9
107,107,"POLYGON ((306503.828 6678260.871, 306504.238 6...",100,107,626,0,626
191,191,"POLYGON ((319431.286 6644823.593, 319625.396 6...",100,191,8100,0,8100
...,...,...,...,...,...,...,...
6005,6005,"POLYGON ((365740.729 6623835.447, 365740.720 6...",100,6005,6049,0,6049
6013,6013,"POLYGON ((365716.437 6624359.046, 365704.944 6...",100,6013,14267,0,14267
6018,6018,"POLYGON ((366058.680 6625112.211, 366058.773 6...",100,6013,550,0,550
6106,6106,"POLYGON ((365032.090 6628922.704, 365032.221 6...",100,6106,1512,0,1512


In [5]:
# school = gpd.read_parquet('./output_data/schools.parquet')
# kindergarten = gpd.read_parquet('./output_data/kindergartens.parquet')
# greening = gpd.read_parquet('./output_data/recreational_areas.parquet')

In [6]:
polygon = gpd.read_file("./output_data/polygon.geojson")
polygon.explore()

In [7]:
gdf = gpd.read_parquet("/Users/mvin/Desktop/masterplanning/output_data/BLOCKS_AND_BUILDINGS_INFO-2.parquet")

In [8]:
def balanc_data(gdf, polygon, school, kindergarten, greening):

    Hectare = 10000
    intersecting_blocks = gpd.overlay(gdf, polygon, how="intersection").drop(columns=["id"])
    intersecting_blocks.rename(columns={"id_1": "id"}, inplace=True)
    gdf = intersecting_blocks

    gdf["current_building_area"] = gdf["current_living_area"] + gdf["current_industrial_area"]
    gdf_ = gdf[
        [
            "block_id",
            "area",
            "current_living_area",
            "current_industrial_area",
            "current_population",
            "current_green_area",
            "floors",
        ]
    ]

    gdf_ = (
        gdf_.merge(school[["id", "population_unprov_schools"]], left_on="block_id", right_on="id")
        .merge(kindergarten[["id", "population_unprov_kindergartens"]], left_on="block_id", right_on="id")
        .merge(greening[["id", "population_unprov_recreational_areas"]], left_on="block_id", right_on="id")
    )
    gdf_.drop(["id_x", "id_y", "id"], axis=1, inplace=True)

    gdf_["area"] = gdf_["area"] / Hectare
    gdf_["current_living_area"] = gdf_["current_living_area"] / Hectare
    gdf_["current_industrial_area"] = gdf_["current_industrial_area"] / Hectare
    gdf_["current_green_area"] = gdf_["current_green_area"] / Hectare

    df_sum = gdf_.sum()
    df_sum["floors"] = gdf_["floors"].mean()
    df_new = pd.DataFrame(df_sum).T

    sample = df_new[df_new["area"] > 10].sample()
    sample = sample.to_dict("records")
    block = sample[0].copy()

    return block

In [9]:
block = balanc_data(gdf=gdf, polygon=polygon, school=school, kindergarten=kindergarten, greening=greening)
block

{'block_id': 140932.0,
 'area': 270.7060423617665,
 'current_living_area': 58.46889635,
 'current_industrial_area': 50.056015810199995,
 'current_population': 49734.0,
 'current_green_area': 17.376400000000004,
 'floors': 3.2708333333333335,
 'population_unprov_schools': 1240.0,
 'population_unprov_kindergartens': 0.0,
 'population_unprov_recreational_areas': 14336.0}

In [10]:
mp = MasterPlan(
    area=block["area"],
    current_living_area=block["current_living_area"],
    current_industrial_area=block["current_industrial_area"],
    current_population=block["current_population"],
    current_green_area=block["current_green_area"],
    current_unprov_schoolkids=block["population_unprov_schools"],
    current_unprov_kids=block["population_unprov_kindergartens"],
    current_unprov_green_population=block["population_unprov_recreational_areas"],
)
updated_block_info = mp.optimal_solution_indicators()
# updated_block_info["block_id"] = '138'
updated_block_info

/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = 

{'area': 270.7060423617665,
 'population': 76009.0,
 'b': 29.99601356107917,
 'green_coef_G': 9.030707660260864,
 'living_area': 137.28342198173553,
 'schools_area': 5.4,
 'schools_capacity': 3300,
 'kindergartens_area': 6.6,
 'kindergartens_capacity': 1680,
 'green_area': 54.0510068790854,
 'G_min_capacity': 90085.01146514234,
 'G_max_capacity': 45042.50573257117,
 'green_coef_G_capacity': 59852.459976014834,
 'op_area': 0.078825,
 'parking1_area': 19.8639,
 'parking2_area': 19.312124999999998}

In [11]:
block

{'block_id': 140932.0,
 'area': 270.7060423617665,
 'current_living_area': 58.46889635,
 'current_industrial_area': 50.056015810199995,
 'current_population': 49734.0,
 'current_green_area': 17.376400000000004,
 'floors': 3.2708333333333335,
 'population_unprov_schools': 1240.0,
 'population_unprov_kindergartens': 0.0,
 'population_unprov_recreational_areas': 14336.0}

In [12]:
updated_block_info

{'area': 270.7060423617665,
 'population': 76009.0,
 'b': 29.99601356107917,
 'green_coef_G': 9.030707660260864,
 'living_area': 137.28342198173553,
 'schools_area': 5.4,
 'schools_capacity': 3300,
 'kindergartens_area': 6.6,
 'kindergartens_capacity': 1680,
 'green_area': 54.0510068790854,
 'G_min_capacity': 90085.01146514234,
 'G_max_capacity': 45042.50573257117,
 'green_coef_G_capacity': 59852.459976014834,
 'op_area': 0.078825,
 'parking1_area': 19.8639,
 'parking2_area': 19.312124999999998}

In [13]:
# for i, row in gdf_.iterrows():
#     sample = gdf_[gdf_['area']>10].sample()
#     sample = sample.to_dict('records')
#     block = sample[0].copy()

#     mp = MasterPlan(area=block['area'],
#                 current_living_area=block['current_living_area'],
#                 current_industrial_area=block['current_industrial_area'],
#                 current_population=block['current_population'],
#                 current_green_area=block['current_green_area'],
#                 current_unprov_schoolkids=block['population_unprov_schools'],
#                 current_unprov_kids=block['population_unprov_kindergartens'],
#                 current_unprov_green_population=block['population_unprov_recreational_areas'])
#     final = mp.optimal_solution_indicators()
#     final["block_id"] = row['block_id']
#     service_gdf.loc[int(updated_block_info["block_id"]), "capacity"] += updated_block_info[f'{service_type}_capacity']